In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [2]:
farms = pd.read_csv("../data-mining/farms.csv")
rubrics = pd.read_csv("../data-mining/rubrics.csv")
stocks = pd.read_csv("../data-mining/stocks.csv")

## Add departement column

In [3]:
def departement_of(code_postal: int) -> int:
    departement = int(str(code_postal)[:2])

    # dom_tom departement code is 3 digits-long instead of 2
    dom_tom: bool = departement == 97
    if dom_tom:
        departement = int(str(code_postal)[:3])

    return departement

farms["departement"] = farms.codePostal.apply(departement_of)

In [4]:
stocks.head()

,raison_sociale,rubric_id,quantite,unite
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq
1,GAEC PORC D'ALGRES,3660,860.0,NaN
2,GAEC PORC D'ALGRES,2102,860.0,NaN
3,GAEC FERME H ET L,2101,105.0,NaN
4,GAEC FERME H ET L,2101,150.0,NaN


In [5]:
farms_stocks = farms.merge(stocks, left_on="name", right_on="raison_sociale")

In [6]:
farms_stocks = farms_stocks[["name", "rubric_id", "quantite", "unite", "departement"]]
farms_stocks

,name,rubric_id,quantite,unite,departement
0,GAEC DES BOURRYS,2102,1069.0,Animaux-Eq,89
1,GAEC PORC D'ALGRES,3660,860.0,NaN,81
2,GAEC PORC D'ALGRES,2102,860.0,NaN,81
3,GAEC FERME H ET L,2101,105.0,NaN,50
4,GAEC FERME H ET L,2101,150.0,NaN,50
...,...,...,...,...,...
23682,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,Animaux-Eq,45
23683,DE LA CLINERIE (GAEC) LEFEVRE Christian,2111,25500.0,NaN,45
23684,DES JOURNETS (SCEA),2102,1857.0,Animaux-Eq,41
23685,EARL DES MUSSETS,2111,40000.0,u,41


In [7]:
farms_stocks = pd.DataFrame({'quantite_mean' : farms_stocks.groupby(
    ["name", "rubric_id", "unite", "departement"])['quantite'].mean()}).reset_index()

In [8]:
farms_stocks

,name,rubric_id,unite,departement,quantite_mean
0,2 L'oiselière (GAEC),2101,u,50,130.000000
1,33 RUE DU BOCAGE (GAEC),2102,Animaux-Eq,50,747.000000
2,4 CANTONS DUBRON (GAEC DES ),2101,u,62,215.000000
3,4 CANTONS DUBRON (GAEC DES ),2102,Animaux-Eq,62,384.000000
4,4 CANTONS DUBRON (GAEC DES ),2111,Animaux-Eq,62,22500.000000
...,...,...,...,...,...
12049,earl la roche marteau,2101,u,36,180.000000
12050,gene +,2102,Animaux-Eq,62,1478.200000
12051,quatres sites (GAEC des),2101,u,19,107.666667
12052,quatres sites (GAEC des),2102,Animaux-Eq,19,830.000000


In [9]:
farms_stocks["departement_unite_rank"] = farms_stocks.groupby(
    ["unite", "departement"])['quantite_mean'].rank()

Example for the departement 44

In [10]:
farms_stocks[farms_stocks["departement"] == 44]

,name,rubric_id,unite,departement,quantite_mean,departement_unite_rank
66,ALLEMAND Didier,2111,u,44,39950.0,97.0
203,BARTHEAU Pierre,2111,Animaux-Eq,44,30000.0,122.0
252,BELAY Mickaël,2102,Animaux-Eq,44,746.0,24.0
285,BERNIER Jean Yves,2111,Animaux-Eq,44,56000.0,134.0
481,BRETHOME Clarisse,2111,Animaux-Eq,44,15000.0,114.5
...,...,...,...,...,...,...
11445,SCEA OCEANAGREE,2102,Animaux-Eq,44,3052.0,105.0
11648,SEGUINEAU Corentin,2101,u,44,480.0,66.0
11744,SOLEAU Florian,2101,u,44,50.0,1.5
11745,SOLEAU Florian,3660,u,44,56100.0,111.0
